# TP2 : Extraction d'informations

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- de se familiariser avec les techniques d’enrichissement (extraction de mots-clés, reconnaissance d’entités nommées, analyse de sentiment...)
- d’appliquer celles-ci au corpus de bulletins de la ville de Bruxelles préalablement convertis au format .txt

Pour ce faire, vous utiliserez différentes librairies Python vues au cours.

## 1. Keywords

### Librairies utilisées dans le code

In [70]:
import os
import textract
from collections import defaultdict
from nltk.corpus import stopwords
import yake

### Création d'une liste de mots à ignorer

In [89]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu"]

sw = set(sw) # set() permet de n'avoir qu'une seule occurrence par mot

print(sw)

ignored = set(["conseil communal", "conseil général", "bruxelles"]) # set = liste avec une occurrence
print(ignored)

{'toi', 'étantes', 'eussiez', 'notre', 'des', 'doit', 'tu', 'moi', 'étée', 'vos', 'dont', 'ces', 'van', 'eussions', 'ta', 'fait', 'soient', 'étais', 'ayante', 'n', 'est', 'aies', 'non', 'serais', 'het', 'l', 'fussiez', 'peut', 'avez', 'contre', 'mes', 'j', 'pas', 'sans', 'auriez', 'lui', 'entre', 'fussent', 'd', 'tout', 'ainsi', 'ait', 'fûtes', 'pendant', 'seront', 'étions', 'par', 'seraient', 'un', 'eus', 'avec', 'de', 'avoir', 'sur', 'es', 'fussions', 'aurions', 'seriez', 'en', 'serons', 'as', 'soyons', 'tous', 'ai', 'trois', 'mon', 'faut', 'suis', 'aurait', 'êtes', 'serai', 'autres', 'sont', 'étant', 'serez', 'eues', 'ayants', 'étiez', 'faire', 'qu', 'jusqu', 'sommes', 'dans', 'ma', 'eux', 'ayantes', 'une', 'vous', 'que', 'aurai', 'bien', 'nos', 'ayez', 'sous', 'eûtes', 'au', 'seras', 'cette', 'ou', 'avait', 'je', 'ne', 'ils', 'votre', 'ses', 'deux', 'eussent', 'les', 'et', 'leur', 'serait', 'aurons', 'aura', 'auraient', 'toutes', 'serions', 'ce', 'sera', 'son', 'elle', 'y', 'encore

### Sélection des fichiers à analyser

#### Stocker la liste des fichiers contenus dans le dossier 'txt'

In [76]:
txt_path = "../data/txt/"
files = sorted(os.listdir(txt_path)) # stocker dans 'files' la liste des fichiers contenus dans le dossier 'txt'
len(files)

2827

#### Lister les fichiers pour Bruxelles et pour Laeken

In [77]:
bxl_1930 = [f for f in files if f.startswith('Bxl_1930')]
print(len(bxl_1930))

lkn_1930 = [f for f in files if f.startswith('Lkn_1930')]
print(len(lkn_30))

31
0


#### Vérification du nombre de fichiers

In [78]:
count_years = defaultdict(int)

for f in files: 
    if "_" in f and f.endswith("txt"): 
        elems = f.split("_") 
        year = elems[1] 
        count_years[year] += 1 
        
n_1930 = count_years['1930']
print(n_1930)

31


### Récupérer le texte des fichiers

#### Créer le dossier adéquat

In [80]:
path_1930 = '../data/1930'
if not os.path.exists(path_1930):
    os.mkdir(path_1930)

#### Copier les fichiers désirés dans le dossier

In [82]:
for txt in bxl_1930:
    text = textract.process(os.path.join(txt_path, txt))
    with open(os.path.join(path_1930, txt), 'wb') as f: 
        f.write(text) 

#### Concaténer le texte des fichiers

In [84]:
!cat ../data/1930/*.txt > ../data/1930.txt

#### Compter le nombre de mots repris dans le fichier

In [93]:
!wc -w ../data/1930.txt

968678 ../data/1930.txt


In [87]:
texts_1930 = open(os.path.join(f"../data/", "1930.txt"), 'r').read()
texts_1930[:500]

'VILLE\n\nDE\n\nBRUXELLES\n\nBULLETIN COMMUNAL\nA N N É E\n\n1930\n\nTOME\n\nII\n\nBRUXELLES\nIMPRIMERIE\n\nE . G U Y O T , S. A.\n\n12. rue Pachéco, 12\n1 930\n\n\x0c\x0cN" l .\n\nCOMTTE RENDU DE LA SÉANCE DU 7 JUILLET IfWO.\n\nV I L L E\n\nD E\n\nBULLETIN\nAnnée\n\nC O N S E I L\nSéance\n\nB R U X E L L E S\n\nGO M M U N AI\n1930.\n\nC O M M U N A L\n\ndu 7 Juillet\n\n1930.\n\nPrésidence de M . ADOLPHE M A X , Bourgmestre.\n\nSOMMAIRE :\nPAGES.\n\n1.\n2.\n3.\n\nCommunications\n. . .\n5\nDémission de M. l\'Echevin Jacqmain\n20\nElection et prestation de serment d'

### Nettoyer le fichier

#### Tokenisation

In [90]:
words = nltk.wordpunct_tokenize(texts_1930)
print(f"{len(words)} words found")

1204041 words found


SyntaxError: invalid syntax (4223200352.py, line 1)

### Extraction des mots clés

#### Instancier l'extracteur de mots clés

In [ ]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
keywords = kw_extractor.extract_keywords(text) # rajouter lower pour enlever la casse 
keywords

#### Ne garder que les bigrammes

In [ ]:
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2 and kw.lower() not in ignored:
        kept.append(kw)
kept

#### Analyse séquentielle en fonction des tomes

In [79]:
this_file = bxl_files[0]
this_file

NameError: name 'bxl_files' is not defined

In [ ]:
for f in sorted(bxl_files)[:10]:
    text = open(os.path.join(data_path, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower()) # rajouter le lower 
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

### Enrichissement de la liste des stopwords

In [ ]:
print(keywords)

## 2. Named Entity Recognition